# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [91]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [69]:
city_vacation_df=pd.read_csv("output_data/cities.csv", encoding="utf-8")
city_vacation_df.head()

,Unnamed: 0,City ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,546105,Nikolskoye,59.70,30.79,33.80,80,90,8.95,RU
1,1,3833367,Ushuaia,-54.80,-68.30,46.40,45,20,31.09,AR
2,2,4030556,Rikitea,-23.12,-134.97,76.08,79,24,7.52,PF
3,3,6170031,Tuktoyaktuk,69.45,-133.04,8.60,84,90,19.46,CA
4,4,5122534,Jamestown,42.10,-79.24,39.99,60,90,26.40,US


In [70]:
city_vacation_df = city_vacation_df.dropna()
city_vacation_df.count()

Unnamed: 0    540
City ID       540
City          540
Lat           540
Long          540
Max Temp      540
Humidity      540
Cloudiness    540
Wind Speed    540
Country       540
dtype: int64

In [71]:
city_vacation_df.head()

,Unnamed: 0,City ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,546105,Nikolskoye,59.70,30.79,33.80,80,90,8.95,RU
1,1,3833367,Ushuaia,-54.80,-68.30,46.40,45,20,31.09,AR
2,2,4030556,Rikitea,-23.12,-134.97,76.08,79,24,7.52,PF
3,3,6170031,Tuktoyaktuk,69.45,-133.04,8.60,84,90,19.46,CA
4,4,5122534,Jamestown,42.10,-79.24,39.99,60,90,26.40,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [72]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Create coordinates
coordinates = city_vacation_df[['Lat','Long']].astype(float)

#Humidity as weight
humidity = city_vacation_df['Humidity'].astype(float)

In [73]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [74]:
#Make copy of dataframe
city_vacation_df.copy().head()

,Unnamed: 0,City ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,546105,Nikolskoye,59.70,30.79,33.80,80,90,8.95,RU
1,1,3833367,Ushuaia,-54.80,-68.30,46.40,45,20,31.09,AR
2,2,4030556,Rikitea,-23.12,-134.97,76.08,79,24,7.52,PF
3,3,6170031,Tuktoyaktuk,69.45,-133.04,8.60,84,90,19.46,CA
4,4,5122534,Jamestown,42.10,-79.24,39.99,60,90,26.40,US


In [75]:
# NO COLD PLACES MAX TEMPS ABOVE 80F
vacation_spots = city_vacation_df.loc[city_vacation_df['Max Temp']>= 80]
vacation_spots.count()

Unnamed: 0    75
City ID       75
City          75
Lat           75
Long          75
Max Temp      75
Humidity      75
Cloudiness    75
Wind Speed    75
Country       75
dtype: int64

In [76]:
# NOT CLOUDY
vacation_spots = vacation_spots.loc[vacation_spots['Cloudiness']<= 40]
vacation_spots.count()

Unnamed: 0    40
City ID       40
City          40
Lat           40
Long          40
Max Temp      40
Humidity      40
Cloudiness    40
Wind Speed    40
Country       40
dtype: int64

In [77]:
# No to humid, but no rain
vacation_spots = vacation_spots.loc[vacation_spots['Humidity']<= 50]
vacation_spots.count()

Unnamed: 0    10
City ID       10
City          10
Lat           10
Long          10
Max Temp      10
Humidity      10
Cloudiness    10
Wind Speed    10
Country       10
dtype: int64

In [78]:
hotel_df = vacation_spots

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [79]:
#Alwayds making sure what my data looks like
hotel_df.head()

,Unnamed: 0,City ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country
20,20,1651810,Airai,-8.93,125.41,82.78,37,2,7.18,TL
61,61,1257771,Saiha,22.48,92.97,85.26,40,0,1.57,IN
100,100,2167426,Emerald,-23.53,148.17,100.99,19,0,10.29,AU
126,126,1254709,Tezu,27.92,96.17,81.01,31,0,3.06,IN
216,216,2070998,Geraldton,-28.77,114.60,82.40,23,0,5.82,AU


In [80]:
#cleaning and organizing hotel_df to add new cloumn Hotel Name types: lodging
hotel_df.drop(['Unnamed: 0', 'City ID'], axis=1)
hotel_df['Hotel Name'] = ''
hotel_df.head()

,Unnamed: 0,City ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
20,20,1651810,Airai,-8.93,125.41,82.78,37,2,7.18,TL,
61,61,1257771,Saiha,22.48,92.97,85.26,40,0,1.57,IN,
100,100,2167426,Emerald,-23.53,148.17,100.99,19,0,10.29,AU,
126,126,1254709,Tezu,27.92,96.17,81.01,31,0,3.06,IN,
216,216,2070998,Geraldton,-28.77,114.60,82.40,23,0,5.82,AU,


In [92]:
#Building google Api, parameters, types: lodging  keyword: hotel

params = {"types":"lodging", "keyword": "hotel", 'radius': 5000, 'key': g_key}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    long = row["Long"]
    params['location'] =f"{lat},{long}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    
    response = requests.get(base_url, params=params).json()
    print(json.dumps(response, indent=4, sort_keys=True))
    

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
{
    "html_attributions": [],
    "next_page_token": "ATtYBwKWLjKJddSc7eMJH2tJFEgZtNFazlc-Ly_reeG7UIeA8VAUhMTRYsSSl9wxihKo96XjckiueXfvVjeuJzv-6hvqgS8h2Ne73Tgo3_krLUsE7TulzGfsZ_4-pGufWpAdJyG5RwcI617P-lCSFim5HEjiW9Jmh4QbLHZSwu1aQ7_CaGleIhdcXax6Z5I4uFauxpGlhfE1n0WybrvYyGxN7ieNecT0QSnME6KeU3StWfzp_vgf_RqWrWVJYSizw7CrI8J4Nmb1kfKGX_Bu-8AAhTFdO2SguA16DVqD7-Bl5Ti6X-GRJMiO9XGYlbcZLmWLZ59kEQuATM-x43hvVPZKxGDhORDacaDH0G-cs_NOO7ABRMqo-KK8FeZQ8ZNKWHZ-pgIzIIU3dDcpONbCgutkZ5PqZcOeffTA4N4SpECccWS9dGL0jhXjvGAM7gOg",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -23.5418436,
                    "lng": 148.1730014
                },
                "viewport": {
                    "northeast": {
                        "lat": -23.54029

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 27.9256949,
                    "lng": 96.16240239999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 27.92706417989272,
                        "lng": 96.16376667989272
                    },
                    "southwest": {
                        "lat": 27.92436452010728,
                        "lng": 96.16106702010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Capital O 66991 Hotel Highway",
            "photos": [
                {
                    "height": 427,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/105326989825922204

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIqmPOAj5t09exb-oAPTgpfvciKYVLs8-KdO4imYAxfvULhNUTqZEGgQCGJZdmz5XWuHI3gX0usLhNnm2T5NS0zWomkMp5JpcDxRSuubxu2zUU8Ydjygy8UHr9qt_rs-UCDQ1CWkh4Cn9_AhWAS1Hh8X4HTbkTxUraxxuxASunp6HcsmykCWYNxBqI4zm6srWju2pAM8tySc_ZlwZAkoGKqbmVuWKMcCW6KjAhyoxKkUisMD4r7VnGexxyh03RKUA8Q2OLAsqL-zasy38qfXCV9QQ7GDQf6oD2vf-JDES63tojNsADK3ORyKGxB_dxXqZYLphOvGRRzMUjbVeSm9bsfvtffnJGwWQxxDHxAdlrgKr8xN2FKlDxfR56sok7DPsIt31SssNR9Z0kr-astKzO9O2ycosa--fueAu7XzbOvoDRUjrpAg8vqWpf6",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -28.7738192,
                    "lng": 114.6082717
                },
                "viewport": {
                    "northeast": {
                        "lat": -28.77237427010728,
                        "lng": 114.6095486298927
                    },
                    "southwest": {
                        "lat": -28.775073929

{
    "html_attributions": [],
    "next_page_token": "ATtYBwI7MJTausWyu8DrDUba9mrmYbR2HQNTpvI5UPD-JcCZfSn-AuSZgcGdubRn4433Wo4KAcYDtWrDHtlQtx2VHy_4CFGJL2U_WKfxFNjph8GEAmjLTg_Xl8rbtmO2CbglrJMCeZvVmaus-NlpKGyhDMFTRugAqOUNKi6yWOFumIiwOV2H6gB1SA6uSRvEKYZ6xByGz_U0QKkHFBM6o7g7JZ8mcd4QILNUbOUdViyQskPcUVWoqQL8Ay2ByPSvdkcW9IF1Fm9oCtWCEqr-FI0-v02qNDrSYFjH-xTM6cRRvUavpTlBsUy_GdSP1xdIkUpMLadTpEmAWRNOIXwDKpK2B5Y_ptVUw-Lj-Fo5X6M2pW8i6sRU57RxXtXPC8BWfqHdtgnh53iNkTTS1jWJyiKxLdkiOdx9SqGMCCwuUR_14bxlBIHUTCbyMp3Xqwj6",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 22.0874076,
                    "lng": 82.1969377
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.08870267989272,
                        "lng": 82.19829457989272
                    },
                    "southwest": {
                        "lat": 22.0860030201072

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIUokXsDjAjAWpUW3wNuphtdB8sdBBvvv4FyHMnIPuXMQOQbFlN4IQnLR4sUM4S_SpOMatrkxeAiGC8KQorpx2gpmLZZoCUrqPQTzSD11awlNvzCC4CXoFIwrRt67AV3u0z_3KkOewz_U5khUTciajKh_hAN211joPkmVoe8s9jRWVl0EQD4JQgv-oaZtVRSKcwJnFNJs_osIwYHT76EAm6PPQNJaQ4Bi5-mlio__ILi55gqsr8RvHoU1bFau6GZfeWUaV0-6dgDbuvi3iLp8d34Z1XpQjSpIDspOr-KA3zgslEJMi0gzYPXPtHF3gu2ZD0jNh_OPrANaj6nUhNQ9qxAJfGYXWV1G5zESZWNOk5obGO1cClSuGnA0nj812e8HgIApuUJimxASEiEec5X_oNdMPPmoBmzbUgXAf3kQcIKNjElA1rUCqizs6M",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 24.5341426,
                    "lng": 72.0260679
                },
                "viewport": {
                    "northeast": {
                        "lat": 24.53549362989272,
                        "lng": 72.02743827989272
                    },
                    "southwest": {
                        "lat": 24.5327939701072

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -37.8296624,
                    "lng": 147.6162127
                },
                "viewport": {
                    "northeast": {
                        "lat": -37.82821862010727,
                        "lng": 147.6175299798927
                    },
                    "southwest": {
                        "lat": -37.83091827989271,
                        "lng": 147.6148303201073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Bairnsdale International",
            "photos": [
                {
                    "height": 3024,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/101783588364719160033\">A

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKoo1rjoWT5KM6Nm6P2B4vOAjMZhbrEQJfZDke78oLEr4sLmhtAyBCbcrKXEOVJ5OV6ekKmfiD7cATqTJkctalUlmQ9sVjQRj3tq7-9yyQSNbqfs-VXvmsTWLmGzNdmzwWVGfjhNRGHRZW9G_nz4xCXA0c1xhddsUayBFTu_7gmJUDXD_nFr5bGadvZwOvy_S9_vbZM_ZiUaF10nZe_7rbv3onddUSvT5XWLB812RsVA5Qt5vXhNZkyu1twshdwjXzGrBO06sbndExJhJ8VDfEWotvCDNdzQuFg1fmQf8ROzcXKdlcMfxbcg0otm0rz8tqO5eIGLkGMu6OndJJ2zRB7a0ia0AyKn18v8WHLE2b-WAI2KaSY2lG5IJBZlf91-Ifdjofi6YIZPYM3DB-HBKSAFLjR3CzOxhRIeZOhLc8PwhTJwe8PdwJouuhW",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -32.7179328,
                    "lng": 152.1044602
                },
                "viewport": {
                    "northeast": {
                        "lat": -32.71658517010727,
                        "lng": 152.1058086798927
                    },
                    "southwest": {
                        "lat": -32.719284829

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.0786577,
                    "lng": 146.2560814
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.07727562010728,
                        "lng": 146.2573498298927
                    },
                    "southwest": {
                        "lat": -20.07997527989272,
                        "lng": 146.2546501701073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Commercial Hotel",
            "photos": [
                {
                    "height": 3024,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/101196110244910094622\">A Google 

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
